In [6]:
import numpy as np
from sklearn.pipeline import Pipeline
from python.utils import load_data
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GroupKFold
from python.features_extraction import ExtractFeatures
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC

In [2]:
fc_parameters = {
    "abs_energy": None,
    "maximum": None,
    "absolute_sum_of_changes": None,
    "fft_coefficient": [{"coeff": 10, "attr": "abs"}],
    "fourier_entropy": [{"bins": 10}],
    "linear_trend": [{"attr": "slope"}],
    "mean": None,
    "number_peaks": [{"n":4}],
    "variance": None,
}

In [3]:
DATA_PATH = 'data'
X_train, y_train, X_test, subjects = load_data(DATA_PATH)

X_train size: (3500, 15872).
y_train size: (3500,).
X_test size: (3500, 15872).


In [7]:
pre_process = Pipeline([
    ("si", SimpleImputer(missing_values=-999999.99, strategy="mean")),
    ("pp", ExtractFeatures(fc_parameters)),
    ("si2", SimpleImputer(missing_values=np.nan, strategy="mean")),
    ("scaler", StandardScaler()),
])

In [8]:
svc_model = Pipeline([
    ("preprocess", pre_process),
    ("svc", OneVsRestClassifier(estimator=SVC(C=0.25, kernel='poly', degree=3, coef0=2, gamma='scale'), n_jobs=2))
])

In [ ]:
cv_res = cross_validate(svc_model, X_train, y_train, groups=subjects, cv=GroupKFold(len(np.unique(subjects))), return_train_score=True, n_jobs=1, scoring="accuracy")
cv_res["test_score"].mean()